**Notas de uso de distintos LLMs**

Llama 3 funciona normal

Gemma 3 funciona normal

GptOSS tiene pedos con la versión actual de transformers

No se probaron más

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from peft import LoraConfig, TaskType

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

openai_genconfig = GenerationConfig.from_pretrained('openai/gpt-oss-20b')
openai_quant_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = torch.bfloat16)
openai_tokenizer = AutoTokenizer.from_pretrained('openai/gpt-oss-20b')
openai = AutoModelForCausalLM.from_pretrained('openai/gpt-oss-20b', torch_dtype = 'auto').to(dev) 
#openai = AutoModelForCausalLM.from_pretrained('openai/gpt-oss-20b', quantization_config = openai_quant_config).to(dev) 

ValueError: The checkpoint you are trying to load has model type `gpt_oss` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [ ]:
gemma_gen_config = GenerationConfig.from_pretrained("google/gemma-3-270m")
gemma_quant_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = torch.bfloat16)

gemma3_270_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")
gemma3_270 = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m", quantization_config = gemma_quant_config).to(dev)

In [2]:
from datasets import load_dataset
kuro_ds = load_dataset('Kurosawama/Translation_DPO_Llama-3.1-8B', split='train')
text = kuro_ds['chosen'][5][0]['content']

In [ ]:
gemma1b_gen_config = GenerationConfig.from_pretrained("google/gemma-3-1b-it")
gemma1b_quant_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = torch.bfloat16)

gemma1b_tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
gemma1b = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it", quantization_config = gemma1b_quant_config).to(dev) 

In [7]:
def generation(text_input, model, tokenizer):
    llm_input = tokenizer(text_input, return_tensors = 'pt', padding = "longest", pad_to_multiple_of = 8).to(dev)
    outputs = model.generate(**llm_input, max_new_tokens = 200)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens = True)[0]
    answer = answer[len(llm_input):]
    print(answer)

In [8]:
generation(text, gemma1b, gemma1b_tokenizer)

   Given a problem description and a question, the task is to parse the problem and the question into first-order logic formulars.
    The grammar of the first-order logic formular is defined as follows:
    1) logical conjunction of expr1 and expr2: expr1 ∧ expr2
    2) logical disjunction of expr1 and expr2: expr1 ∨ expr2
    3) logical exclusive disjunction of expr1 and expr2: expr1 ⊕ expr2
    4) logical negation of expr1: ¬expr1
    5) expr1 implies expr2: expr1 → expr2
    6) expr1 if and only if expr2: expr1 ↔ expr2
    7) logical universal quantification: ∀x
    8) logical existential quantification: ∃x
    A detailed example is shown next. Only answer with the premises, omit the explanation.
    --------------
    Problem:
    All people who regularly drink coffee are dependent on caffeine. People either regularly drink coffee or joke about being addicted to caffeine. No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. Rina is either a student

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

In [8]:
test = """
∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
    ∀x (Drinks(x) ⊕ Jokes(x)) ::: People either regularly drink coffee or joke about being addicted to caffeine.
    ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
    (Student(rina) ∧ Unaware(rina)) ⊕ ¬(Student(rina) ∨ Unaware(rina)) ::: Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. 
    ¬(Dependent(rina) ∧ Student(rina)) → (Dependent(rina) ∧ Student(rina)) ⊕ ¬(Dependent(rina) ∨ Student(rina)) ::: If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    """
print(len(test))

836
